In [67]:
import os
import math
import numpy as np
from matplotlib import pyplot as plt
import torch
# import gpytorch
# from gpytorch import kernels, means, models, mlls, settings
# from gpytorch import distributions as distr
import pandas as pd
%matplotlib inline

train_size = 25000
test_size = 10000
print(torch.cuda.is_available())

True


生成训练数据

In [33]:
data = pd.read_csv('../records/initial.csv')
data.drop('Unnamed: 0',axis = 1,inplace = True)
data.head()
data2 = data.apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x)))
data2.result = data.result
data2.to_csv('../records/normalized.csv')

In [68]:
train = data.iloc[:train_size,: ]
test = data.iloc[train_size:, :]
train_x = data.iloc[0:train_size,:-1]
train_y = data.iloc[0:train_size,-1]
test_x = data.iloc[train_size:train_size+test_size, :-1]
test_y = data.iloc[train_size:train_size+test_size, -1]

In [6]:
train_x = torch.from_numpy(np.array(train_x))
train_y = torch.from_numpy(np.array(train_y))
test_x = torch.from_numpy(np.array(test_x))
test_y = torch.from_numpy(np.array(test_y))


模型建立

In [69]:
import torch
# print(torch.cuda.is_available())
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

In [70]:
import torch.nn.functional as F
import torch.optim as optim
from torch import nn


In [82]:
train_batch_size = 64
test_batch_size = 32
num_epoches = 20
lr = 0.5
momentum = 0.9
#归一化

normal_train = train.apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x)))
normal_train.result = train.result
normal_test = test.apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x)))
normal_test.result = test.result

class PilotDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        """
        纸飞机dataset
        :param data_dir: Dataframe
        :param transform: torch.transform，数据预处理
        """
        self.label_name = {"1": 0, "100": 1}
        self.data_info = data_dir
        self.transform = transform

    def __getitem__(self, index):
        now = self.data_info.iloc[index, :]
        X = now[:-1]
        y = now[-1]
        # X = X.apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x)))
        # print(X)
        X = np.array(X)
        y = np.array(y)
        X = X.astype(np.float32)
        y = y.astype(np.float32)
        X = torch.tensor(X)
        y = torch.tensor(y)
        if self.transform is not None:
            X = self.transform(X) 
        
        
        return X,y

    def __len__(self):
        return len(self.data_info)
#previous transform
# transform = transforms.Compose([transforms.Normalize([0.5],[0.5])])
#download the dataset

train_dataset = PilotDataset(normal_train)
test_dataset = PilotDataset(normal_test)

# declare dataloader 
train_loader = DataLoader(train_dataset, batch_size=train_batch_size,
                          shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)


,Weight,Elevators,theta,rou,result
0,48,46,35,105,44.0
1,9,42,41,95,42.8
2,87,36,38,110,9.8
3,50,15,50,85,41.8
4,100,71,41,110,46.3
...,...,...,...,...,...
24995,39,42,41,115,16.4
24996,5,28,48,80,46.4
24997,87,42,41,105,42.4
24998,99,22,41,105,11.9


In [84]:
class Net(nn.Module):
    
    def __init__(self, in_dim, n_hidden_1,n_hidden_2, n_hidden_3,n_hidden_4,out_dim):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_dim, n_hidden_1),nn.BatchNorm1d(n_hidden_1), nn.Dropout(0.2))
        self.layer2 = nn.Sequential(nn.Linear(n_hidden_1, n_hidden_2),nn.BatchNorm1d(n_hidden_2),nn.Dropout(0.2))
        self.layer3 = nn.Sequential(nn.Linear(n_hidden_2, n_hidden_3),nn.BatchNorm1d(n_hidden_3))
        self.layer4 = nn.Sequential(nn.Linear(n_hidden_3, n_hidden_4),nn.BatchNorm1d(n_hidden_4))
        self.layer5 = nn.Sequential(nn.Linear(n_hidden_4, out_dim),nn.BatchNorm1d(out_dim))
        
    def forward(self,x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        x = (self.layer5(x))
        return x

device = torch.device("cuda:0" if torch.cuda.is_available() is True else "cpu")

In [85]:
model = Net(4, 8, 16, 16, 4,1)
model.to(device)

#declare Loss Function and optimizer

criterion = torch.nn.MSELoss()
# optimizer = optim.SGD(model.parameters(),lr,momentum)
optimizer = optim.Adam(model.parameters(),lr)

In [ ]:
## Train Model

losses = []
acces = []
eval_losses = []
eval_acces= []

for epoch in range(num_epoches):
    train_loss = 0
    train_acc = 0
    model.train()
    
    #动态修改学习率参数
    if epoch%5 == 0:
        optimizer.param_groups[0]['lr'] *= 0.1
    for X,label in train_loader:
        X = X.to(device)
        label = label.to(device)
        #forward
        out = model(X)
        loss = criterion(out,label)
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() #loss是个标量，在pytorch里用item取出这个唯一的元素
        #calculate the accuracy of this training
        # _, pred = torch.max(out,1) #按行取最大值 也就是预测的那个数
        outcome = abs(out-label) <= 0.1
        num_correct =outcome.sum().item()
        acc = num_correct/X.shape[0]
        train_acc+=acc
        
    losses.append(train_loss/ len(train_loader))
    acces.append(train_acc/ len(train_loader))
    
    eval_loss = 0
    eval_acc = 0
    
    model.eval() #Model change to eval mode
    for img,label in test_loader:
        img=img.to(device)
        label = label.to(device)
        # img = img.view(img.size(0),-1)
        
        out = model(img)
        loss = criterion(out,label)
        eval_loss+=loss.item()
        
        # _,pred = out.max(1)
        num_cor = (abs(out-label) <= 0.2).sum().item()
        acc = num_cor/img.shape[0]
        eval_acc+=acc
        
    eval_losses.append(eval_loss/len(test_loader))
    eval_acces.append(eval_acc / len(test_loader))
    print("epoch:{}, Train loss: {:.4f}, Train acc: {:.4f}, Test loss: {:.4f}. Test acc:{:.4f}\n".format(epoch, train_loss/len(train_loader), train_acc/len(train_loader), eval_loss/len(test_loader), eval_acc/len(test_loader)))


In [29]:
print(train_x.shape[0]*train_x.shape[1])
print(len(train_x))

60000
15000


In [68]:
for X,y in train_loader:
    X = X.to(device)
    y = y.to(device)
    out = model(X)
    here = (abs(out-y) <= 0.01)
    # print(X)
    break



IndexingError: Too many indexers

In [88]:
model.eval()
now = np.array([[0.90,0.62,0.533333333,0.866666667
]])
now = now.astype(np.float32)
now = torch.tensor(now).to(device)
out = model(now)
print(out)

tensor([[29.7431]], device='cuda:0', grad_fn=<CudnnBatchNormBackward>)


In [81]:
for layer,param in model.state_dict().items(): # param is weight or bias(Tensor) 
	print (layer,param)

layer1.0.weight tensor([[-0.1704, -0.5773, -0.1374,  0.7331],
        [ 0.8228,  0.4425, -0.1597, -0.2100],
        [-0.2074, -0.0989, -0.1212, -0.7224],
        [ 0.1347,  0.6153, -0.0672,  0.0894],
        [-0.7915,  0.0261,  0.0704, -0.5685],
        [-0.6218,  0.0359,  0.0531,  0.2521],
        [ 0.5029, -0.1751,  0.3792, -0.4011],
        [-0.1148, -0.1068, -0.6935, -0.1060]], device='cuda:0')
layer1.0.bias tensor([-0.4764, -0.1074,  0.3954, -0.4913, -0.4690,  0.3869, -0.2075, -0.4838],
       device='cuda:0')
layer1.1.weight tensor([0.5731, 1.3303, 1.5022, 1.0008, 1.3142, 0.5325, 1.4713, 0.5870],
       device='cuda:0')
layer1.1.bias tensor([ 0.0934,  0.1074,  0.2824,  0.1254,  0.1056, -0.5303,  0.3865, -0.4560],
       device='cuda:0')
layer1.1.running_mean tensor([-0.3594,  0.2620, -0.3071, -0.1538, -1.2174,  0.2824, -0.0977, -1.0151],
       device='cuda:0')
layer1.1.running_var tensor([0.0314, 0.0694, 0.0185, 0.0150, 0.0685, 0.0349, 0.0370, 0.0145],
       device='cuda:0')
la

In [87]:

for layer,param in model.state_dict().items(): # param is weight or bias(Tensor) 
	print (layer,param)

layer1.0.weight tensor([[-0.0160,  0.5516,  0.4051, -0.6863],
        [-0.8919,  0.0289,  0.4211, -0.2373],
        [ 0.2575,  0.3322, -0.0543, -0.4241],
        [-0.5136, -0.2577, -0.1332,  0.2500],
        [-0.4385,  0.3830, -0.0396, -0.0650],
        [ 0.0504,  0.1444,  0.3351, -0.6784],
        [-0.0125, -0.7866,  0.0588, -0.2121],
        [ 0.2830,  0.1172,  0.8781, -0.1618]], device='cuda:0')
layer1.0.bias tensor([ 0.1227,  0.0983,  0.2334, -0.0382, -0.1132,  0.0991,  0.3353,  0.0331],
       device='cuda:0')
layer1.1.weight tensor([0.4929, 0.6888, 0.9110, 1.2355, 0.6976, 1.4557, 0.6501, 1.4008],
       device='cuda:0')
layer1.1.bias tensor([-0.5180, -0.1785, -0.3469,  0.1871, -0.4024,  0.1308, -0.3546,  0.4826],
       device='cuda:0')
layer1.1.running_mean tensor([ 0.0704, -0.2927,  0.1774, -0.3002, -0.2521, -0.1100, -0.0953,  0.5665],
       device='cuda:0')
layer1.1.running_var tensor([0.0356, 0.0808, 0.0138, 0.0295, 0.0201, 0.0252, 0.0218, 0.0374],
       device='cuda:0')
la